In [1]:
import pandas as pd
import os

# 1. 파일 경로 설정
file_path = '서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv'

# 2. CSV 파일 로드
try:
    df_raw = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df_raw = pd.read_csv(file_path, encoding='cp949')

print("✅ CSV 파일 로드 성공")

# 3. 데이터 전처리 (Transform)
# (1) 컬럼명 매핑
rename_map = {
    '고유역번호(외부역코드)': 'stnCd',
    '역명': 'stnNm',
    '위도': 'lat',
    '경도': 'lon'
}
df_raw.rename(columns=rename_map, inplace=True)

# (2) 호선명 포맷 통일 (1 -> 1호선)
# 기존 DB의 lineNm 컬럼과 조인하기 위해 필수적인 단계입니다.
if '호선' in df_raw.columns:
    df_raw['lineNm'] = df_raw['호선'].astype(str).apply(lambda x: x + '호선' if x.isdigit() else x)
else:
    df_raw['lineNm'] = '정보없음'

# (3) 역코드 문자열 변환 (DB 타입 매칭)
df_raw['stnCd'] = df_raw['stnCd'].astype(str).str.strip()

# (4) 최종 컬럼 선택
target_cols = ['stnCd', 'stnNm', 'lineNm', 'lat', 'lon']
df_final = df_raw[target_cols].copy()

# 4. 결과 확인 (상위 5개)
print("\n📊 [변환 결과 미리보기] station_meta 테이블 적재용")
print("="*60)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

display(df_final.head(5))

print("-" * 60)
print(f"총 데이터 건수: {len(df_final)}개")
print("이 형태 그대로 DB에 적재하면 '역코드(stnCd)'를 기준으로 조인할 수 있습니다.")

✅ CSV 파일 로드 성공

📊 [변환 결과 미리보기] station_meta 테이블 적재용


,stnCd,stnNm,lineNm,lat,lon
0,150,서울,1호선,37.553150,126.972533
1,151,시청,1호선,37.563590,126.975407
2,152,종각,1호선,37.570203,126.983116
3,153,종로3가,1호선,37.570429,126.992095
4,154,종로5가,1호선,37.570971,127.001900


------------------------------------------------------------
총 데이터 건수: 276개
이 형태 그대로 DB에 적재하면 '역코드(stnCd)'를 기준으로 조인할 수 있습니다.


In [2]:
import pandas as pd
import dbconnect
import os
import sys

# ---------------------------------------------------------
# 1. 설정 및 파일 로드
# ---------------------------------------------------------
FILE_NAME = '서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv'

def load_and_transform_csv():
    """CSV 파일을 읽어 DB 적재 포맷으로 변환"""
    if not os.path.exists(FILE_NAME):
        print(f"❌ 파일을 찾을 수 없습니다: {FILE_NAME}")
        sys.exit()

    print(f"📂 파일 로드 중... ({FILE_NAME})")
    try:
        df = pd.read_csv(FILE_NAME, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(FILE_NAME, encoding='cp949')

    # (1) 컬럼명 매핑
    rename_map = {
        '고유역번호(외부역코드)': 'stnCd',
        '역명': 'stnNm',
        '위도': 'lat',
        '경도': 'lon'
    }
    df.rename(columns=rename_map, inplace=True)

    # (2) 호선명 변환 (예: 1 -> 1호선)
    # 호선 정보가 없거나 비어있을 경우 대비
    if '호선' in df.columns:
        df['lineNm'] = df['호선'].astype(str).apply(lambda x: x + '호선' if x.isdigit() else x)
    else:
        print("⚠️ '호선' 컬럼이 없어 '정보없음'으로 처리합니다.")
        df['lineNm'] = '정보없음'

    # (3) 데이터 타입 정리 (DB 스키마 매칭)
    df['stnCd'] = df['stnCd'].astype(str).str.strip()
    df['stnNm'] = df['stnNm'].astype(str).str.strip()
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

    # (4) 결측치 제거 (좌표가 없는 경우 제외)
    df_clean = df.dropna(subset=['lat', 'lon', 'stnCd'])
    
    # (5) 최종 컬럼 선택
    target_cols = ['stnCd', 'stnNm', 'lineNm', 'lat', 'lon']
    return df_clean[target_cols]

# ---------------------------------------------------------
# 2. DB 적재 함수
# ---------------------------------------------------------
def insert_meta_data(conn, df):
    """변환된 데이터를 station_meta 테이블에 Insert/Update"""
    if df.empty:
        print("⚠️ 적재할 데이터가 없습니다.")
        return

    cursor = conn.cursor()
    
    # Upsert 쿼리: 이미 존재하는 역(stnCd 기준)이면 좌표와 이름을 최신으로 업데이트
    sql = """
        INSERT INTO station_meta (stnCd, stnNm, lineNm, lat, lon)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            stnNm = VALUES(stnNm),
            lineNm = VALUES(lineNm),
            lat = VALUES(lat),
            lon = VALUES(lon),
            created_at = NOW()
    """

    # DataFrame을 튜플 리스트로 변환
    data_tuples = [tuple(x) for x in df.to_numpy()]
    
    try:
        cursor.executemany(sql, data_tuples)
        conn.commit()
        print(f"✅ DB 적재 성공! (총 {len(data_tuples)}건 처리)")
    except Exception as e:
        print(f"❌ DB 적재 실패: {e}")
        conn.rollback()

# ---------------------------------------------------------
# 3. 메인 실행
# ---------------------------------------------------------
if __name__ == "__main__":
    # 1. 데이터 준비
    df_result = load_and_transform_csv()
    print(f"   -> 변환 완료: {len(df_result)}행")

    # 2. DB 연결
    print("\n🔌 DB 연결 시도...")
    conn = dbconnect.MydbConnect('seoul_urban_lab')

    # 3. 적재 실행
    insert_meta_data(conn, df_result)

    # 4. 연결 종료
    conn.close()
    print("\n🎉 역사 좌표 마스터 데이터 구축 완료.")

📂 파일 로드 중... (서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv)
   -> 변환 완료: 276행

🔌 DB 연결 시도...
데이터베이서 연결 시작!
MySQL DB 연결 성공
✅ DB 적재 성공! (총 276건 처리)

🎉 역사 좌표 마스터 데이터 구축 완료.


In [3]:
import pandas as pd
import dbconnect
import os
import sys

# ---------------------------------------------------------
# 1. 설정 및 파일 로드
# ---------------------------------------------------------
FILE_NAME = '서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv'

def load_and_transform_csv_v2():
    if not os.path.exists(FILE_NAME):
        print(f"❌ 파일을 찾을 수 없습니다: {FILE_NAME}")
        sys.exit()

    print(f"📂 파일 로드 및 포맷 교정 중... ({FILE_NAME})")
    try:
        df = pd.read_csv(FILE_NAME, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(FILE_NAME, encoding='cp949')

    # (1) 컬럼명 매핑
    rename_map = {
        '고유역번호(외부역코드)': 'stnCd',
        '역명': 'stnNm',
        '위도': 'lat',
        '경도': 'lon'
    }
    df.rename(columns=rename_map, inplace=True)

    # (2) 호선명 변환 (1 -> 1호선)
    if '호선' in df.columns:
        df['lineNm'] = df['호선'].astype(str).apply(lambda x: x + '호선' if x.isdigit() else x)
    else:
        df['lineNm'] = '정보없음'

    # ---------------------------------------------------------
    # 🚨 [핵심 수정 사항] 역코드 4자리 패딩 (Zero Padding)
    # ---------------------------------------------------------
    # API 데이터는 '0150' 형식이므로, CSV의 '150'을 '0150'으로 맞춰줍니다.
    df['stnCd'] = df['stnCd'].astype(str).str.strip().str.zfill(4)
    
    # stnNm 공백 제거
    df['stnNm'] = df['stnNm'].astype(str).str.strip()

    # 좌표 데이터 숫자 변환
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

    # 결측치 제거
    df_clean = df.dropna(subset=['lat', 'lon', 'stnCd'])
    
    target_cols = ['stnCd', 'stnNm', 'lineNm', 'lat', 'lon']
    return df_clean[target_cols]

# ---------------------------------------------------------
# 2. DB 적재 함수
# ---------------------------------------------------------
def insert_meta_data(conn, df):
    if df.empty:
        return

    cursor = conn.cursor()
    
    # 이미 들어간 3자리 코드 데이터가 있다면 덮어쓰거나, 
    # 깔끔하게 하기 위해 TRUNCATE(비우기) 후 다시 넣는 것을 추천합니다.
    # 여기서는 ON DUPLICATE KEY UPDATE를 사용합니다.
    sql = """
        INSERT INTO station_meta (stnCd, stnNm, lineNm, lat, lon)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            stnNm = VALUES(stnNm),
            lineNm = VALUES(lineNm),
            lat = VALUES(lat),
            lon = VALUES(lon),
            created_at = NOW()
    """
    
    data_tuples = [tuple(x) for x in df.to_numpy()]
    
    try:
        cursor.executemany(sql, data_tuples)
        conn.commit()
        print(f"✅ DB 적재 성공! (총 {len(data_tuples)}건)")
        
        # 1호선 샘플 확인
        print("\n🔍 [검증] 1호선(서울역) 코드가 '0150'으로 잘 들어갔는지 확인:")
        cursor.execute("SELECT * FROM station_meta WHERE stnNm = '서울역' LIMIT 1")
        print(cursor.fetchone())
        
    except Exception as e:
        print(f"❌ DB 적재 실패: {e}")
        conn.rollback()

if __name__ == "__main__":
    # 데이터 변환
    df_result = load_and_transform_csv_v2()
    
    # 미리보기 확인
    print(f"   -> 변환된 1호선 코드 예시: {df_result.iloc[0]['stnCd']} (4자리여야 함)")

    # DB 연결 및 적재
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    insert_meta_data(conn, df_result)
    conn.close()

📂 파일 로드 및 포맷 교정 중... (서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv)
   -> 변환된 1호선 코드 예시: 0150 (4자리여야 함)
데이터베이서 연결 시작!
MySQL DB 연결 성공
✅ DB 적재 성공! (총 276건)

🔍 [검증] 1호선(서울역) 코드가 '0150'으로 잘 들어갔는지 확인:
None


In [4]:
import pandas as pd
import dbconnect
import os
import sys

# ---------------------------------------------------------
# 1. 설정 및 파일 로드
# ---------------------------------------------------------
FILE_NAME = '서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv'

def load_original_csv():
    if not os.path.exists(FILE_NAME):
        print(f"❌ 파일을 찾을 수 없습니다: {FILE_NAME}")
        sys.exit()

    print(f"📂 원본 파일 로드 중... ({FILE_NAME})")
    try:
        df = pd.read_csv(FILE_NAME, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(FILE_NAME, encoding='cp949')

    # (1) 컬럼명 매핑
    rename_map = {
        '고유역번호(외부역코드)': 'stnCd',
        '역명': 'stnNm',
        '위도': 'lat',
        '경도': 'lon'
    }
    df.rename(columns=rename_map, inplace=True)

    # (2) 호선명 변환 (1 -> 1호선)
    # 호선 명칭은 '1호선'으로 유지해야 조인이 되므로 이 로직은 남겨둡니다.
    if '호선' in df.columns:
        df['lineNm'] = df['호선'].astype(str).apply(lambda x: x + '호선' if x.isdigit() else x)
    else:
        df['lineNm'] = '정보없음'

    # ---------------------------------------------------------
    # 🔄 [원상 복구] 역코드 0 채우기(zfill) 제거
    # ---------------------------------------------------------
    # CSV에 있는 그대로 '150' 형태로 변환합니다.
    df['stnCd'] = df['stnCd'].astype(str).str.strip()
    
    # 공백 제거 및 숫자 변환
    df['stnNm'] = df['stnNm'].astype(str).str.strip()
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

    # 결측치 제거
    df_clean = df.dropna(subset=['lat', 'lon', 'stnCd'])
    
    target_cols = ['stnCd', 'stnNm', 'lineNm', 'lat', 'lon']
    return df_clean[target_cols]

# ---------------------------------------------------------
# 2. DB 초기화 및 재적재 함수
# ---------------------------------------------------------
def reset_and_insert(conn, df):
    if df.empty:
        return

    cursor = conn.cursor()
    
    try:
        # 1. 테이블 초기화 (기존 데이터 삭제)
        print("🧹 기존 station_meta 테이블을 비우는 중 (TRUNCATE)...")
        cursor.execute("TRUNCATE TABLE station_meta")
        conn.commit()
        
        # 2. 데이터 적재
        print("📥 원본 데이터 적재 시작...")
        sql = """
            INSERT INTO station_meta (stnCd, stnNm, lineNm, lat, lon)
            VALUES (%s, %s, %s, %s, %s)
        """
        data_tuples = [tuple(x) for x in df.to_numpy()]
        cursor.executemany(sql, data_tuples)
        conn.commit()
        
        print(f"✅ 원상 복구 완료! (총 {len(data_tuples)}건)")
        
        # 검증
        print("\n🔍 [검증] 서울역 코드가 '150'으로 돌아왔는지 확인:")
        cursor.execute("SELECT * FROM station_meta WHERE stnNm = '서울역' LIMIT 1")
        print(cursor.fetchone())
        
    except Exception as e:
        print(f"❌ 작업 실패: {e}")
        conn.rollback()

if __name__ == "__main__":
    # 데이터 준비
    df_result = load_original_csv()
    
    # DB 연결 및 실행
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    reset_and_insert(conn, df_result)
    conn.close()

📂 원본 파일 로드 중... (서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv)
데이터베이서 연결 시작!
MySQL DB 연결 성공
🧹 기존 station_meta 테이블을 비우는 중 (TRUNCATE)...
📥 원본 데이터 적재 시작...
✅ 원상 복구 완료! (총 276건)

🔍 [검증] 서울역 코드가 '150'으로 돌아왔는지 확인:
None


In [5]:
import pandas as pd
import dbconnect
import sys

def diagnose_join_issue():
    print("🕵️‍♂️ [진단 시작] 1~4호선 데이터 매칭 실패 원인 분석\n")
    
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    cursor = conn.cursor()

    try:
        # ---------------------------------------------------------
        # 1. API 승하차 테이블(subway_traffic_log)의 1호선 코드 확인
        # ---------------------------------------------------------
        print("1️⃣ [subway_traffic_log] 승객 데이터의 1호선 역 코드 샘플:")
        sql_log = """
            SELECT DISTINCT lineNm, stnCd, stnNm 
            FROM subway_traffic_log 
            WHERE lineNm = '1호선' 
            LIMIT 5
        """
        df_log = pd.read_sql(sql_log, conn)
        if df_log.empty:
            print("   ⚠️ 승객 데이터에 1호선 데이터가 없습니다.")
        else:
            print(df_log.to_string(index=False))
            # 데이터 타입 확인을 위해 따옴표로 감싸서 출력
            sample_code = df_log.iloc[0]['stnCd']
            print(f"   👉 실제 저장된 값: '{sample_code}' (길이: {len(sample_code)})")

        print("-" * 50)

        # ---------------------------------------------------------
        # 2. 좌표 마스터 테이블(station_meta)의 1호선 코드 확인
        # ---------------------------------------------------------
        print("2️⃣ [station_meta] 좌표 데이터의 1호선 역 코드 샘플:")
        sql_meta = """
            SELECT DISTINCT lineNm, stnCd, stnNm 
            FROM station_meta 
            WHERE lineNm = '1호선' 
            LIMIT 5
        """
        df_meta = pd.read_sql(sql_meta, conn)
        if df_meta.empty:
            print("   ⚠️ 좌표 데이터에 1호선 데이터가 없습니다.")
        else:
            print(df_meta.to_string(index=False))
            sample_code = df_meta.iloc[0]['stnCd']
            print(f"   👉 실제 저장된 값: '{sample_code}' (길이: {len(sample_code)})")

        print("-" * 50)

        # ---------------------------------------------------------
        # 3. 조인(JOIN) 테스트 결과
        # ---------------------------------------------------------
        print("3️⃣ [결론] 두 테이블 조인 시도 결과:")
        sql_join = """
            SELECT COUNT(*) 
            FROM subway_traffic_log t
            JOIN station_meta m ON t.stnCd = m.stnCd
            WHERE t.lineNm = '1호선'
        """
        cursor.execute(sql_join)
        count = cursor.fetchone()[0]
        
        if count == 0:
            print(f"   ❌ 매칭된 데이터 건수: {count}건")
            print("   💡 원인: 한쪽은 '0150'(4자리), 다른 쪽은 '150'(3자리)라서 컴퓨터는 서로 다른 문자로 인식합니다.")
            print("   💡 해결책: station_meta 테이블의 코드를 다시 4자리('0150')로 맞춰줘야 합니다.")
        else:
            print(f"   ✅ 매칭된 데이터 건수: {count}건 (정상)")

    except Exception as e:
        print(f"❌ 진단 중 오류 발생: {e}")
        
    finally:
        conn.close()

if __name__ == "__main__":
    diagnose_join_issue()

🕵️‍♂️ [진단 시작] 1~4호선 데이터 매칭 실패 원인 분석

데이터베이서 연결 시작!
MySQL DB 연결 성공
1️⃣ [subway_traffic_log] 승객 데이터의 1호선 역 코드 샘플:
lineNm stnCd        stnNm
   1호선  0159          동묘앞
   1호선  0158 청량리(서울시립대입구)
   1호선  0157          제기동
   1호선  0156          신설동
   1호선  0155          동대문
   👉 실제 저장된 값: '0159' (길이: 4)
--------------------------------------------------
2️⃣ [station_meta] 좌표 데이터의 1호선 역 코드 샘플:
lineNm stnCd stnNm
   1호선   150    서울
   1호선   151    시청
   1호선   152    종각
   1호선   153  종로3가
   1호선   154  종로5가
   👉 실제 저장된 값: '150' (길이: 3)
--------------------------------------------------
3️⃣ [결론] 두 테이블 조인 시도 결과:
   ❌ 매칭된 데이터 건수: 0건
   💡 원인: 한쪽은 '0150'(4자리), 다른 쪽은 '150'(3자리)라서 컴퓨터는 서로 다른 문자로 인식합니다.
   💡 해결책: station_meta 테이블의 코드를 다시 4자리('0150')로 맞춰줘야 합니다.


C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\2594537732.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_log = pd.read_sql(sql_log, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\2594537732.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_meta = pd.read_sql(sql_meta, conn)


In [6]:
import pandas as pd
import dbconnect
import sys

def fix_station_codes():
    print("🛠️ [데이터 동기화] subway_traffic_log의 역 코드를 수정합니다 (0150 -> 150)...")
    
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    cursor = conn.cursor()

    try:
        # ---------------------------------------------------------
        # 1. 수정 대상 데이터 확인 (업데이트 전)
        # ---------------------------------------------------------
        print("1️⃣ 변경 전 상태 확인 (1호선 샘플):")
        check_sql = "SELECT DISTINCT stnCd, stnNm FROM subway_traffic_log WHERE stnCd LIKE '015%' LIMIT 3"
        cursor.execute(check_sql)
        before_rows = cursor.fetchall()
        for row in before_rows:
            print(f"   - {row}")

        if not before_rows:
            print("   (이미 '0'이 제거되었거나 대상 데이터가 없습니다.)")

        # ---------------------------------------------------------
        # 2. UPDATE 쿼리 실행 (핵심)
        # ---------------------------------------------------------
        # LEADING '0' FROM stnCd : 앞에 있는 0만 제거 (중간의 0은 유지)
        print("\n2️⃣ UPDATE 쿼리 실행 중...")
        
        update_sql = """
            UPDATE subway_traffic_log 
            SET stnCd = TRIM(LEADING '0' FROM stnCd)
            WHERE stnCd LIKE '0%';
        """
        cursor.execute(update_sql)
        affected_rows = cursor.rowcount
        conn.commit()
        
        print(f"   ✅ 업데이트 완료! 총 {affected_rows}건의 데이터가 수정되었습니다.")

        # ---------------------------------------------------------
        # 3. 변경 후 조인(Join) 테스트
        # ---------------------------------------------------------
        print("\n3️⃣ 변경 후 조인 테스트 (subway_traffic_log ↔ station_meta):")
        
        # 1호선 서울역(150) 기준으로 잘 붙는지 확인
        test_join_sql = """
            SELECT t.stnNm, t.lineNm, t.stnCd as Traffic_Code, m.stnCd as Meta_Code, m.lat, m.lon
            FROM subway_traffic_log t
            JOIN station_meta m ON t.stnCd = m.stnCd
            WHERE t.stnNm = '서울역' AND t.lineNm = '1호선'
            LIMIT 1
        """
        
        cursor.execute(test_join_sql)
        result = cursor.fetchone()
        
        if result:
            print(f"   🎉 매칭 성공! \n   {result}")
            print("   이제 Streamlit 지도에서 1호선이 정상적으로 보일 것입니다.")
        else:
            print("   ❌ 여전히 매칭되지 않습니다. 코드를 다시 확인해야 합니다.")

    except Exception as e:
        print(f"❌ 작업 중 오류 발생: {e}")
        conn.rollback()
        
    finally:
        conn.close()

if __name__ == "__main__":
    fix_station_codes()

🛠️ [데이터 동기화] subway_traffic_log의 역 코드를 수정합니다 (0150 -> 150)...
데이터베이서 연결 시작!
MySQL DB 연결 성공
1️⃣ 변경 전 상태 확인 (1호선 샘플):
   - ('0159', '동묘앞')
   - ('0158', '청량리(서울시립대입구)')
   - ('0157', '제기동')

2️⃣ UPDATE 쿼리 실행 중...
   ✅ 업데이트 완료! 총 124709건의 데이터가 수정되었습니다.

3️⃣ 변경 후 조인 테스트 (subway_traffic_log ↔ station_meta):
   🎉 매칭 성공! 
   ('서울역', '1호선', '150', '150', Decimal('37.5531500'), Decimal('126.9725330'))
   이제 Streamlit 지도에서 1호선이 정상적으로 보일 것입니다.


In [7]:
import pandas as pd
import os
import sys
import dbconnect  # dbconnect.py 모듈 활용

# ---------------------------------------------------------
# 1. 설정: 내보낼 테이블 리스트와 저장 경로
# ---------------------------------------------------------
OUTPUT_DIR = "backup_csv"  # 저장할 폴더명

# (테이블명, 저장할 파일명) 튜플 리스트
TARGET_TABLES = [
    ("subway_traffic_log", "subway_traffic_log_latest.csv"),
    ("subway_traffic_log_senior_22-24", "subway_traffic_senior_22_24.csv")
]

def export_data():
    # 저장 폴더 생성 (없으면 생성)
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"📂 백업 폴더 생성 완료: {OUTPUT_DIR}/")

    print("🔌 DB 연결 시도...")
    conn = dbconnect.MydbConnect('seoul_urban_lab')

    try:
        for table_name, file_name in TARGET_TABLES:
            print(f"\n🔄 [Export] '{table_name}' 내보내는 중...")
            
            # -----------------------------------------------------
            # SQL 쿼리 작성 (특수문자 포함 테이블명 처리)
            # -----------------------------------------------------
            # 테이블명에 하이픈(-)이 있으므로 반드시 백틱(`)으로 감싸야 합니다.
            sql = f"SELECT * FROM `{table_name}`"
            
            # -----------------------------------------------------
            # 데이터 조회 (pandas read_sql 활용)
            # -----------------------------------------------------
            try:
                # chunksize를 설정하면 메모리 부족을 방지할 수 있으나, 
                # 현재 규모에서는 한 번에 불러와도 무방합니다.
                df = pd.read_sql(sql, conn)
                
                if df.empty:
                    print(f"   ⚠️ 경고: 테이블이 비어있습니다.")
                    continue
                
                # -----------------------------------------------------
                # CSV 저장
                # -----------------------------------------------------
                # encoding='utf-8-sig': 엑셀에서 한글이 깨지지 않게 함
                save_path = os.path.join(OUTPUT_DIR, file_name)
                df.to_csv(save_path, index=False, encoding='utf-8-sig')
                
                print(f"   ✅ 저장 완료! ({len(df)}행)")
                print(f"      -> 경로: {save_path}")
                
            except Exception as e:
                print(f"   ❌ 테이블 조회 실패: {e}")

    except Exception as e:
        print(f"❌ 작업 중 치명적 오류: {e}")
        
    finally:
        conn.close()
        print("\n" + "="*50)
        print("🎉 모든 내보내기 작업이 완료되었습니다.")
        print("="*50)

if __name__ == "__main__":
    export_data()

📂 백업 폴더 생성 완료: backup_csv/
🔌 DB 연결 시도...
데이터베이서 연결 시작!
MySQL DB 연결 성공

🔄 [Export] 'subway_traffic_log' 내보내는 중...


C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\4003510457.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


   ✅ 저장 완료! (262044행)
      -> 경로: backup_csv\subway_traffic_log_latest.csv

🔄 [Export] 'subway_traffic_log_senior_22-24' 내보내는 중...
   ✅ 저장 완료! (5965194행)
      -> 경로: backup_csv\subway_traffic_senior_22_24.csv

🎉 모든 내보내기 작업이 완료되었습니다.


In [8]:
import pandas as pd
import dbconnect
import sys

def find_ghost_stations():
    print("🕵️‍♂️ [추적 시작] 지도에서 사라진 4개 역을 찾습니다...\n")
    
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    
    try:
        # ---------------------------------------------------------
        # 1. 기준 데이터 (Meta) 가져오기 - 276개
        # ---------------------------------------------------------
        sql_meta = "SELECT stnCd, stnNm, lineNm FROM station_meta"
        df_meta = pd.read_sql(sql_meta, conn)
        meta_set = set(df_meta['stnCd'])
        
        print(f"✅ 기준(Meta) 역 개수: {len(df_meta)}개")

        # ---------------------------------------------------------
        # 2. 최근 승객 데이터 (API 로그)와 비교
        # ---------------------------------------------------------
        print("\n🔍 [1단계] API 승객 데이터(subway_traffic_log)와 비교")
        sql_log = "SELECT DISTINCT stnCd FROM subway_traffic_log"
        df_log = pd.read_sql(sql_log, conn)
        log_set = set(df_log['stnCd'])
        
        # 차집합 연산 (Meta에는 있는데 Log에는 없는 것)
        missing_in_log = meta_set - log_set
        
        if missing_in_log:
            print(f"   ⚠️ 총 {len(missing_in_log)}개 역이 승객 데이터에 없습니다.")
            print("   👉 아래 역들은 트래픽 데이터가 없어서 지도에 안 나옵니다:")
            
            # 상세 정보 출력
            missing_df = df_meta[df_meta['stnCd'].isin(missing_in_log)]
            print(missing_df.to_string(index=False))
        else:
            print("   ✨ 모든 역이 정상적으로 매칭됩니다.")

        # ---------------------------------------------------------
        # 3. 노인 데이터 (22-24년)와 비교
        # ---------------------------------------------------------
        print("\n🔍 [2단계] 노인 데이터(subway_traffic_log_senior_22-24)와 비교")
        # 테이블명에 백틱(`) 필수
        sql_senior = "SELECT DISTINCT stnCd FROM `subway_traffic_log_senior_22-24`"
        df_senior = pd.read_sql(sql_senior, conn)
        senior_set = set(df_senior['stnCd'])
        
        missing_in_senior = meta_set - senior_set
        
        if missing_in_senior:
            print(f"   ⚠️ 총 {len(missing_in_senior)}개 역이 노인 데이터에 없습니다.")
            # 상세 정보 출력 (상위 10개만)
            missing_senior_df = df_meta[df_meta['stnCd'].isin(missing_in_senior)]
            print(missing_senior_df.head(10).to_string(index=False))
        else:
            print("   ✨ 모든 역이 정상적으로 매칭됩니다.")

    except Exception as e:
        print(f"❌ 분석 중 오류 발생: {e}")
        
    finally:
        conn.close()

if __name__ == "__main__":
    find_ghost_stations()

🕵️‍♂️ [추적 시작] 지도에서 사라진 4개 역을 찾습니다...

데이터베이서 연결 시작!
MySQL DB 연결 성공
✅ 기준(Meta) 역 개수: 276개

🔍 [1단계] API 승객 데이터(subway_traffic_log)와 비교
   ⚠️ 총 1개 역이 승객 데이터에 없습니다.
   👉 아래 역들은 트래픽 데이터가 없어서 지도에 안 나옵니다:
stnCd stnNm lineNm
  200   까치산    2호선

🔍 [2단계] 노인 데이터(subway_traffic_log_senior_22-24)와 비교


C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\3871461358.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_meta = pd.read_sql(sql_meta, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\3871461358.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_log = pd.read_sql(sql_log, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\3871461358.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_senior = pd.read_sql(sql_senior, conn)


   ⚠️ 총 1개 역이 노인 데이터에 없습니다.
stnCd stnNm lineNm
  200   까치산    2호선


In [9]:
import pandas as pd
import dbconnect

def check_missing_on_latest_date():
    print("🕵️‍♂️ [정밀 진단] 지도에 표시되는 '최신 날짜' 기준으로 누락된 역을 찾습니다...\n")
    
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    
    try:
        # 1. 최신 날짜 확인
        cursor = conn.cursor()
        cursor.execute("SELECT MAX(pasngDe) FROM subway_traffic_log")
        latest_date = cursor.fetchone()[0]
        print(f"📅 분석 기준 날짜 (최신): {latest_date}")
        
        # 2. 기준(Meta) 역 코드 가져오기
        sql_meta = "SELECT stnCd, stnNm, lineNm FROM station_meta"
        df_meta = pd.read_sql(sql_meta, conn)
        meta_set = set(df_meta['stnCd'])
        
        # 3. 최신 날짜의 승객 데이터 가져오기
        sql_log = f"SELECT DISTINCT stnCd FROM subway_traffic_log WHERE pasngDe = '{latest_date}'"
        df_log = pd.read_sql(sql_log, conn)
        log_set = set(df_log['stnCd'])
        
        # 4. 차집합 (Meta에는 있는데, 오늘자 Log에는 없는 것)
        missing_codes = meta_set - log_set
        
        print(f"\n📊 분석 결과: 총 {len(missing_codes)}개 역이 누락되었습니다.")
        
        if len(missing_codes) > 0:
            print("👇 [누락된 역 명단]")
            missing_df = df_meta[df_meta['stnCd'].isin(missing_codes)]
            pd.set_option('display.max_rows', None)
            display(missing_df)
            
            print("\n💡 [원인 분석]")
            print("1. '까치산(2호선)'은 애초에 로그에 코드가 없어서 누락되었습니다.")
            print("2. 나머지 역들은 '주말/공휴일'이라 운행하지 않았거나, 해당 날짜에만 집계가 안 된 경우입니다.")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        
    finally:
        conn.close()

if __name__ == "__main__":
    check_missing_on_latest_date()

🕵️‍♂️ [정밀 진단] 지도에 표시되는 '최신 날짜' 기준으로 누락된 역을 찾습니다...

데이터베이서 연결 시작!
MySQL DB 연결 성공
📅 분석 기준 날짜 (최신): 20260201

📊 분석 결과: 총 3개 역이 누락되었습니다.
👇 [누락된 역 명단]


C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\129252472.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_meta = pd.read_sql(sql_meta, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4116\129252472.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_log = pd.read_sql(sql_log, conn)


,stnCd,stnNm,lineNm
60,200,까치산,2호선
214,2615,봉화산,6호선
215,2649,신내,6호선



💡 [원인 분석]
1. '까치산(2호선)'은 애초에 로그에 코드가 없어서 누락되었습니다.
2. 나머지 역들은 '주말/공휴일'이라 운행하지 않았거나, 해당 날짜에만 집계가 안 된 경우입니다.


In [1]:
import pandas as pd
import dbconnect

def identify_missing_details():
    print("🕵️‍♂️ [심층 분석] 사라진 3개 역의 정체를 밝힙니다...\n")
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    
    try:
        # 1. 최신 날짜(어제) 확인
        cursor = conn.cursor()
        cursor.execute("SELECT MAX(pasngDe) FROM subway_traffic_log")
        latest_date = cursor.fetchone()[0]
        
        # 2. 메타 데이터 (276개)
        df_meta = pd.read_sql("SELECT stnCd, stnNm, lineNm FROM station_meta", conn)
        
        # 3. 어제 로그 데이터
        df_log_today = pd.read_sql(f"SELECT DISTINCT stnCd FROM subway_traffic_log WHERE pasngDe = '{latest_date}'", conn)
        
        # 4. 전체 역사 로그 데이터 (과거 포함)
        df_log_history = pd.read_sql("SELECT DISTINCT stnCd FROM subway_traffic_log", conn)
        
        # 5. [범인 색출]
        meta_set = set(df_meta['stnCd'])
        today_set = set(df_log_today['stnCd'])
        history_set = set(df_log_history['stnCd'])
        
        missing_today = meta_set - today_set # 오늘 없는 3개
        
        print(f"📅 분석 날짜: {latest_date}")
        print(f"📊 오늘 누락된 역: 총 {len(missing_today)}개\n")
        
        # 상세 분석 루프
        results = []
        for code in missing_today:
            info = df_meta[df_meta['stnCd'] == code].iloc[0]
            
            # 과거에는 있었나?
            is_in_history = code in history_set
            
            status = ""
            if not is_in_history:
                status = "❌ 아예 데이터 없음 (영구 결번)"
            else:
                status = "💤 오늘은 쉼 (평일엔 있음)"
                
            results.append({
                '역코드': code,
                '역명': info['stnNm'],
                '호선': info['lineNm'],
                '상태': status
            })
            
        # 결과 출력
        df_result = pd.DataFrame(results)
        print(df_result.to_string(index=False))
        
        print("\n" + "="*50)
        print("💡 [분석가 코멘트]")
        
        # 272개 vs 273개 미스터리 해결
        expected_cnt = 276 - len(missing_today)
        print(f"   - 이론상 지도에 찍혀야 할 역 개수: {expected_cnt}개")
        print("   - 만약 Streamlit에서 1개가 더 적게(272개) 보인다면,")
        print("     혹시 '연신내(6호선)'이나 '신내' 처럼 특정 역의 좌표가 겹쳐있거나,")
        print("     데이터베이스 조회 시점 차이일 수 있습니다.")

    except Exception as e:
        print(f"❌ 오류: {e}")
        
    finally:
        conn.close()

if __name__ == "__main__":
    identify_missing_details()

🕵️‍♂️ [심층 분석] 사라진 3개 역의 정체를 밝힙니다...

데이터베이서 연결 시작!
MySQL DB 연결 성공
📅 분석 날짜: 20260201
📊 오늘 누락된 역: 총 3개

 역코드  역명  호선                  상태
 200 까치산 2호선 ❌ 아예 데이터 없음 (영구 결번)
2649  신내 6호선    💤 오늘은 쉼 (평일엔 있음)
2615 봉화산 6호선    💤 오늘은 쉼 (평일엔 있음)

💡 [분석가 코멘트]
   - 이론상 지도에 찍혀야 할 역 개수: 273개
   - 만약 Streamlit에서 1개가 더 적게(272개) 보인다면,
     혹시 '연신내(6호선)'이나 '신내' 처럼 특정 역의 좌표가 겹쳐있거나,
     데이터베이스 조회 시점 차이일 수 있습니다.


C:\Users\Admin\AppData\Local\Temp\ipykernel_26600\2732310019.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_meta = pd.read_sql("SELECT stnCd, stnNm, lineNm FROM station_meta", conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_26600\2732310019.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_log_today = pd.read_sql(f"SELECT DISTINCT stnCd FROM subway_traffic_log WHERE pasngDe = '{latest_date}'", conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_26600\2732310019.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchem

In [2]:
import pandas as pd
import dbconnect
import os

def export_station_meta():
    print("📤 [Export] station_meta 테이블을 CSV로 내보냅니다...")
    
    # 1. DB 연결
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    
    try:
        # 2. 데이터 조회 쿼리
        sql = "SELECT * FROM station_meta ORDER BY lineNm, stnCd"
        
        # 3. 데이터프레임으로 변환
        df = pd.read_sql(sql, conn)
        
        if df.empty:
            print("⚠️ 테이블이 비어있습니다.")
            return

        # 4. CSV 저장 설정
        # 파일명: station_meta_backup_YYYYMMDD.csv 형태 권장
        file_name = "station_meta_backup.csv"
        
        # utf-8-sig: 엑셀에서 한글 깨짐 방지
        df.to_csv(file_name, index=False, encoding='utf-8-sig')
        
        print(f"\n✅ 내보내기 완료!")
        print(f"   - 파일명: {file_name}")
        print(f"   - 저장 경로: {os.path.abspath(file_name)}")
        print(f"   - 총 데이터: {len(df)}건")
        
        # 미리보기
        print("\n[데이터 미리보기]")
        print(df.head())

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        
    finally:
        conn.close()

if __name__ == "__main__":
    export_station_meta()

📤 [Export] station_meta 테이블을 CSV로 내보냅니다...
데이터베이서 연결 시작!
MySQL DB 연결 성공

✅ 내보내기 완료!
   - 파일명: station_meta_backup.csv
   - 저장 경로: c:\Users\Admin\Desktop\hdc2\0202 프로젝트 구축\station_meta_backup.csv
   - 총 데이터: 276건

[데이터 미리보기]
   id stnCd stnNm lineNm        lat         lon          created_at
0   1   150    서울    1호선  37.553150  126.972533 2026-02-02 14:43:16
1   2   151    시청    1호선  37.563590  126.975407 2026-02-02 14:43:16
2   3   152    종각    1호선  37.570203  126.983116 2026-02-02 14:43:16
3   4   153  종로3가    1호선  37.570429  126.992095 2026-02-02 14:43:16
4   5   154  종로5가    1호선  37.570971  127.001900 2026-02-02 14:43:16


C:\Users\Admin\AppData\Local\Temp\ipykernel_26600\3519676216.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
